In [5]:
import tensorflow as tf

from tensorflow import keras


In [6]:
from tensorflow.keras import layers


In [7]:
from tensorflow.keras import mixed_precision

In [8]:
print('Sana')

Sana


In [9]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0


In [10]:
# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')

In [11]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [12]:
inputs = keras.Input(shape=(784,), name='digits')
if tf.config.list_physical_devices('GPU'):
  print('The model will run with 4096 units on a GPU')
  num_units = 4096
else:
  # Use fewer units on CPUs so the model finishes in a reasonable amount of time
  print('The model will run with 64 units on a CPU')
  num_units = 64
dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

The model will run with 4096 units on a GPU


In [13]:
print(dense1.dtype_policy)
print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

<Policy "mixed_float16">
x.dtype: float16
dense1.kernel.dtype: float32


In [14]:
# INCORRECT: softmax and model output will be float16, when it should be float32
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float16


In [15]:
# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Outputs dtype: float32


In [16]:
# The linear activation is an identity function. So this simply casts 'outputs'
# to float32. In this particular case, 'outputs' is already float32 so this is a
# no-op.
outputs = layers.Activation('linear', dtype='float32')(outputs)

In [19]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
import numpy as np

x_train=np.load('mnist/x_train.npy', allow_pickle=True) 
y_train=np.load('mnist/y_train.npy', allow_pickle=True) 
x_test=np.load('mnist/x_test.npy', allow_pickle=True) 
y_test=np.load('mnist/y_test.npy', allow_pickle=True) 


x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255


In [20]:
initial_weights = model.get_weights()

In [21]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/5
6/6 [==============================] - 5s 177ms/step - loss: 4.9611 - accuracy: 0.2673 - val_loss: 0.7782 - val_accuracy: 0.8143
Epoch 2/5
6/6 [==============================] - 0s 30ms/step - loss: 0.6684 - accuracy: 0.8144 - val_loss: 0.3218 - val_accuracy: 0.9085
Epoch 3/5
6/6 [==============================] - 0s 30ms/step - loss: 0.3681 - accuracy: 0.8820 - val_loss: 0.2773 - val_accuracy: 0.9174
Epoch 4/5
6/6 [==============================] - 0s 29ms/step - loss: 0.2541 - accuracy: 0.9250 - val_loss: 0.3652 - val_accuracy: 0.8742
Epoch 5/5
6/6 [==============================] - 0s 30ms/step - loss: 0.3670 - accuracy: 0.8746 - val_loss: 0.1663 - val_accuracy: 0.9528
313/313 - 0s - loss: 0.1687 - accuracy: 0.9486
Test loss: 0.16874361038208008
Test accuracy: 0.9485999941825867


In [22]:
x = tf.constant(256, dtype='float16')
(x ** 2).numpy()  # Overflow

inf

In [23]:
x = tf.constant(1e-5, dtype='float16')
(x ** 2).numpy()  # Underflow

0.0

In [24]:
loss_scale = 1024
loss = model(inputs)
loss *= loss_scale
# Assume `grads` are float32. You do not want to divide float16 gradients.
grads = compute_gradient(loss, model.trainable_variables)
grads /= loss_scale

NameError: name 'compute_gradient' is not defined